# Matrix and vector computation
In this notebook we'll see how matrix computation is performed in Python with an example from statistical mechanics: the diffusion equation, also known as random walk in finance. Next we will implement the same algorithm using NumPy.

In [3]:
import time

We might look carefully at the diffusion equation to better understand what it means

$$ \frac{\partial u(u, t)}{\partial t} = D \nabla^2 u(x, t)$$

where D is the diffusion coefficient that we assume constant. In one dimension, e.g. x, it is written

$$\frac{\partial u(u, t)}{\partial t} = D \frac{\partial^2 u(x, t)}{\partial x^2}$$

If we use the Euler's method to discretize the partial derivatives we have

$$\frac{u(x, t + \Delta t)}{\Delta t} = D \frac{u(u + \Delta x, t) - u(x - \Delta x, t) - 2u(x, t)}{\Delta x^2}$$

This means that in order to compute the value of u at x for the next time step $t + \Delta t$ we need to know the value of u in the two neighboring grid points at $x - \Delta x$ and at $x + \Delta x$. Once we know the initial value at each grid point we can compute the future value of u at any point in space and at any future time step.  

We want to compute the values of the diffusion equation for all the grid points for a given number of steps. The grid is instantiated as a list array and then it is initialized  

In [18]:
def init_grid(grid_shape):
    # grid instantiation
    grid = [[0.0 for x in range(grid_shape[1])] for x in range(grid_shape[0])]
    
    # grid initialization
    block_low = int(grid_shape[0] * 0.4)
    block_high = int(grid_shape[0] * 0.5)
    for i in range(block_low, block_high):
        for j in range(block_low, block_high):
            grid[i][j] = 0.005
    return grid

The *diffuse()* function computes the next value of diffusion equation for each grid point using the Euler's discretization method starting from the values in the input grid.

In [29]:
#@profile
def diffuse(grid, dt, D=1.0):
    xmax, ymax = grid_shape
    new_grid = [[0.0 for x in range(grid_shape[1])] for x in range(grid_shape[0])]
    for i in range(xmax):
        for j in range(ymax):
            grid_xx = (
                grid[(i + 1) % xmax][j] + grid[(i - 1) % xmax][j] - 2.0 * grid[i][j]
            )
            grid_yy = (
                grid[i][(j + 1) % ymax] + grid[i][(j - 1) % ymax] - 2.0 * grid[i][j]
            )
            new_grid[i][j] = grid[i][j] + D * (grid_xx + grid_yy) * dt
    return new_grid

The *evolve_grid* function computes the diffuse equation for a number of iteration

In [30]:
def evolve_grid(grid, num_iterations):
    # grid evolution
    start = time.time()
    for i in range(num_iterations):
        grid = diffuse(grid, 0.1)
    return time.time() - start

In [31]:
if __name__ == "__main__":
    number_of_steps = 500
    grid_shape = (640, 640)
    grid = init_grid(grid_shape)
    t = evolve_grid(grid, number_of_steps)
    print('execution time: ', t)

execution time:  264.1395628452301
